# Time Series Project

## Business Understanding 

This project is a a regression project on time series data. A store in Ecuador, Favorita wants to be able to make prediction on sales based on all important factors across all its stores. 
This requires preparing existing data into format that enables us to create a good model to make good predictions in the future. It also requires an analysis to get clarity on some features of the data gathered and how they come together to influence outcomes.

## Data Understanding 


### Questions 

1. How are the clusters performing against each other? 
2. What family brings in the most money across all stores?
3. Which locales are the busiest in terms of transactions?
4. In which year did most of the transactions happen?
5. How have sales been over the years?
6. What are the dates of the top 10 sales?
7. What were the dates of the highest sales for each year?
8. How many transactions happen in the christmas period(20th - 31st December) each year?

### Hypothesis 

#### Null: Oil prices do not have an effect on sales 
#### Alternate: Oil prices have an effect on sales

### Data Handling 
### Import necessary libraries

In [1]:
# Data Handling
import pandas as pd 
import numpy as np 

# Visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import mplcursors
import seaborn as sns

# Encoding
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


from sklearn.model_selection import train_test_split
# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error


### Load Data

In [2]:
train = pd.read_csv("train.csv")
transactions = pd.read_csv("transactions.csv")
holiday = pd.read_csv("holiday.csv")
oil  = pd.read_csv("oil.csv")
stores = pd.read_csv("stores.csv")
test = pd.read_excel('test.xlsx')

### Get an overview of the data frames

In [3]:
# train dataframe
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
# transactions dataframe
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Train Dataframe

- The training data contains information about dates, store numbers, product families, promotions, and sales figures. 
- The data shows sales and time series data for the features store_nbr, family, and onpromotion.
- The store_nbr feature indicates the specific location where the goods are sold by the retailer. 
- The family feature describes the category of goods being sold. 
- The sales column provides the total sales for a particular product family at a specific retailer on a given day. - - - Sales can involve fractional units so decimal values are possible. 
- The onpromotion column indicates the total number of promoted products within a family at a specific store on a specific date.

In [5]:
# holiday dataframe 
holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### Holiday Dataframe 

- Shows events and holidays, their descriptions and whether or not it's transferred

In [6]:
# oil dataframe 
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


### Oil Dataframe 
- Oil prices play a crucial role in how things are purchased in Ecuador
- This dataframe shows the oil prices per barrel based on dates

In [7]:
# stores dataframe 
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Stores Dataframe 
- Keeps information on various stores 
- Contains information such as store type, city, cluster and state the store is located 
- Cluster refers to a grouping of related stores 

In [8]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


### Test Dataframe
- Contains same information as train dataframe
- Contains information that will be used to test the model yet to be trained

## Understand the features/columns of the dataframes

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [10]:
# Change date column to date time 
train['date'] = pd.to_datetime(train['date'])

In [11]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [12]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [13]:
# Change date column to date time 
transactions['date'] = pd.to_datetime(transactions['date'])

In [14]:
transactions.dtypes

date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object

In [15]:
holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [16]:
# Change date column to date time 
holiday['date'] = pd.to_datetime(holiday['date'])

In [17]:
holiday.dtypes

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
dtype: object

In [18]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [19]:
# Change date column to date time 
oil['date'] = pd.to_datetime(oil['date'])

In [20]:
oil.dtypes

date          datetime64[ns]
dcoilwtico           float64
dtype: object

In [21]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


In [23]:
# Change date column to date time 
test['date'] = pd.to_datetime(test['date'])

In [24]:
test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
onpromotion             int64
dtype: object

### Check for the shape of all dataframes 

In [25]:
train.shape

(3000888, 6)

In [26]:
transactions.shape

(83488, 3)

In [27]:
holiday.shape

(350, 6)

In [28]:
oil.shape

(1218, 2)

In [29]:
stores.shape

(54, 5)

In [30]:
test.shape

(28512, 5)

## Data Preparation

### Check for missing values 

In [31]:
train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [32]:
transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [33]:
holiday.isna().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [34]:
oil.isna().sum()

date           0
dcoilwtico    43
dtype: int64

In [35]:
# look at the numeric values in oil df
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190001
25%,46.405001
50%,53.189999
75%,95.660000
max,110.620003


In [36]:
oil['dcoilwtico']

0             NaN
1       93.139999
2       92.970001
3       93.120003
4       93.199997
          ...    
1213    47.650002
1214    46.400002
1215    46.459999
1216    45.959999
1217    47.259998
Name: dcoilwtico, Length: 1218, dtype: float64

The missing values in the oil dataframe have to be filled. Since the range between minimum and maximum values is very large and the values progress or decrease steadily it will be good to use the forward fill technique

In [37]:
oil['dcoilwtico'].unique()

array([         nan,  93.13999939,  92.97000122,  93.12000275,
        93.19999695,  93.20999908,  93.08000183,  93.80999756,
        93.59999847,  94.26999664,  93.26000214,  94.27999878,
        95.48999786,  95.61000061,  96.08999634,  95.05999756,
        95.34999847,  95.15000153,  95.94999695,  97.62000275,
        97.98000336,  97.65000153,  97.45999908,  96.20999908,
        96.68000031,  96.44000244,  95.83999634,  95.70999908,
        97.01000214,  97.48000336,  97.02999878,  97.30000305,
        96.69000244,  94.91999817,  92.79000092,  92.73999786,
        92.62999725,  92.83999634,  92.02999878,  90.70999908,
        90.12999725,  90.87999725,  90.47000122,  91.52999878,
        92.01000214,  92.06999969,  92.44000244,  92.47000122,
        93.02999878,  93.48999786,  93.70999908,  92.45999908,
        93.41000366,  94.55000305,  95.98999786,  96.52999878,
        97.23999786,  97.09999847,  97.23000336,  95.01999664,
        92.76000214,  93.36000061,  94.18000031,  94.58

In [38]:
# Replace nan value with np.nan
oil.replace('nan', np.nan, inplace=True)

In [39]:
# Fill null values with forward fill
oil = oil.fillna(method = 'ffill')

In [40]:
oil.isna().sum()

date          0
dcoilwtico    1
dtype: int64

In [41]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [42]:
# single missing value is at the start of the dataframe so fill with backward fill 
oil = oil.fillna(method = 'bfill')

In [43]:
oil.isna().sum()

date          0
dcoilwtico    0
dtype: int64

In [44]:
stores.isna().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [45]:
test.isna().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

### Make Date the index in dataframes

In [46]:
# train dataframe 
train.set_index('date', inplace=True)

In [47]:
train.head(2)

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0
2013-01-01,1,1,BABY CARE,0.0,0


In [48]:
# transactions dataframe 
transactions.set_index('date', inplace=True)

In [49]:
transactions.head(2)

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111


In [50]:
# holiday dataframe 
holiday.set_index('date', inplace=True)

In [51]:
holiday.head(2)

,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [52]:
# oil dataframe 
oil.set_index('date', inplace=True)

In [53]:
oil.head(2)

,dcoilwtico
date,
2013-01-01,93.139999
2013-01-02,93.139999


In [54]:
# test dataframe 
test.set_index('date', inplace=True)

In [55]:
test.head(2)

,id,store_nbr,family,onpromotion
date,,,,
2017-08-16,3000888,1,AUTOMOTIVE,0
2017-08-16,3000889,1,BABY CARE,0


### Merge the dataframes

In [ ]:
# Merge the train and transactions dataframes based on the date and store number columns
merged = pd.merge(train, transactions, on=['date', 'store_nbr'])

In [ ]:
merged.head()

In [ ]:
# Merge the merged dataframe with the holiday dataframe based on the date column
merged = pd.merge(merged, holiday, on='date')

In [ ]:
merged.head()

In [ ]:
# Merge the merged data with the oil dataframe on the date column
merged = pd.merge(merged, oil, on='date')

In [ ]:
merged.head()

- The date column has been set us the index of the merged dataframe
- This means it will be lost when merged with the stores dataframe on the store_nbr column
- To solve this the index has to be reset

In [ ]:
# Reset index
merged.reset_index(inplace=True)

In [ ]:
merged.info()

In [ ]:
# Merge the merged data with the stores dataframe on the stores number column
train = pd.merge(merged, stores, on='store_nbr')

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train.shape

### Rename columns to aid understanding
Some column names may not clearly describe the data in them so they need to be renamed

In [ ]:
train.info()

In [ ]:
train.head()

- Change type_x column name to holiday_type to reflect what it is in the Holiday dataframe 
- Change 'dcoilwtico' column name to reflect what it represents ie. oil price 
- Change type_y column name to store_type to reflect what it represents in the stores dataframe 

In [ ]:
# Changing column names
train = train.rename(columns= {'type_x': 'holiday_type', 'dcoilwtico': 'oil_price', 'type_y':'store_type' })

In [ ]:
# Confirm the change 
train.columns

### Preparing data in Merged Dataframe 

In [ ]:
# Checking for missing values 
train.isnull().sum()

In [ ]:
# Checking for duplicates 
train.duplicated().sum()

In [ ]:
# Get summary of numerical values
train.describe()

### Questions 

#### 1. How are the clusters performing against each other? 

In [ ]:
train['cluster'].unique()

In [ ]:
# Show the sales per cluster
cluster_sales = train.groupby('cluster')['sales'].sum().reset_index()
cluster_sales 

In [ ]:
# Plot the sales per Cluster

# Setting figure size
fig, ax = plt.subplots(figsize=(10, 10))

sns.barplot(x='cluster', y='sales', data=cluster_sales, ax=ax)
ax.set_xlabel('Cluster')
ax.set_ylabel('Total Sales')
ax.set_title('Total Sales per Cluster')


# Converting y-axis values to full figures
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))


plt.show()

From the plot above we can tell that Cluster 14 is the highest performing cluster and Cluster 16 is the worst performing in terms of sum of sales 

#### 2. What family brings in the most money across all stores?

In [ ]:
# Finding the unique families
train['family'].unique()

In [ ]:
# Find the sales per family
family_sales = train.groupby('family')['sales'].sum().reset_index()
family_sales

In [ ]:
# Plot the sales per family

# Setting figure size
fig, ax = plt.subplots(figsize=(10, 10))

sns.barplot(x='sales', y='family', data=family_sales, ax=ax)
ax.set_xlabel('Total Sales')
ax.set_ylabel('Family')
ax.set_title('Total Sales per Family')
plt.xticks(rotation=45)

# Converting x-axis values to full figures
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))


plt.show()

From the plot above we can tell that GROCERY I is the highest selling 'family' out of the 33 'families'

#### 3. Which locales are the busiest in terms of transactions?

In [ ]:
# Find unique locales
train['locale_name'].unique()

In [ ]:
# Group transactions by the unique locale_names
locale_trans = train.groupby('locale_name')['transactions'].sum().reset_index()
locale_trans

In [ ]:
# Plotting the locale names against the number of transactions 
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(x='locale_name', y='transactions', data=locale_trans, ax=ax)
ax.set_xlabel('Locale Name')
ax.set_ylabel('Transactions')
ax.set_title('Transactions per Locale')

# Rotate x-axis labels
plt.xticks(rotation=70)

# Converting y-axis values to full figures
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))

plt.show()

From the above graph we can tell that the locale_name 'Ecuador' had the most transactions

### 4. In which year did most of the transactions happen?

In [ ]:
# Create array to store years from date column
train['Year'] = train['date'].dt.year
train['Year'].unique()

In [ ]:
# Group transactions by the years
transactions_per_year = train.groupby('Year')['transactions'].count()
transactions_per_year

In [ ]:
plt.figure(figsize=(8, 8))  # Set the figure size if needed
sns.set(style="whitegrid")

plt.pie(transactions_per_year, labels=transactions_per_year.index, autopct='%1.1f%%')
plt.title('Distribution of Transactions per Year')

plt.show()

From the graph above we can tell that the year 2016 had the most transactions in the dataset

#### 5. How have sales been over the years?

In [ ]:
# Create array to store years from date column
train['Year'] = train['date'].dt.year
train['Year'].unique()

In [ ]:
# Group sales by the years
sales_per_year = train.groupby('Year')['sales'].sum().reset_index()
sales_per_year

In [ ]:
# Plotting Sales per year
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='sales', data=sales_per_year, marker='o')
plt.xlabel('Year')
plt.ylabel('Sales')
plt.title('Sales per Year')

# Removing grid lines
plt.grid(False)

# Changing x-axis values to integers
plt.xticks(sales_per_year['Year'], sales_per_year['Year'].astype(int))

# Converting y-axis values to full figures
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x):,}'))

plt.show()

#### 6. What are the dates of the top 10 sales?

In [ ]:
# Finding the top 10 sales
top_10_sales = train.nlargest(10, 'sales')
top_10_sales

In [ ]:
# Plotting the top 10 sales dates
plt.figure(figsize=(10, 6))
sns.lineplot(x='date', y='sales', data=top_10_sales, marker='o')

plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Dates of Top 10 Sales')
plt.xticks(rotation=45)

plt.grid(False)
plt.show()

#### 7. What were the dates of the highest sales for each year?

In [ ]:
# Finding the highest sales per each year 
highest_sales_per_year = train.groupby(train['date'].dt.year).apply(lambda x: x.loc[x['sales'].idxmax()])
highest_sales_per_year

In [ ]:
# Finding the highest dates of the highest sales per each year
highest_sales_date_per_year = highest_sales_per_year[['date', 'sales']].reset_index(drop=True)
highest_sales_date_per_year

In [ ]:
# Plotting the highest sales dates per year

plt.figure(figsize=(10, 6))
sns.barplot(x='date', y='sales', data=highest_sales_date_per_year)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Highest Sales Dates per Year')

plt.xticks(rotation=45)

plt.show()


#### 8. How many transactions happen in the christmas period(20th - 31st December) each year?

In [ ]:
# Finding the number of transactions per christmas period
christmas_transactions = train[train['date'].dt.day.between(20, 31) & train['date'].dt.month.eq(12)].groupby(train['date'].dt.year)['transactions'].count()
christmas_transactions

In [ ]:
x_values = np.arange(len(christmas_transactions))
plt.figure(figsize=(10, 6))
plt.plot(x_values, christmas_transactions.values, marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('Transactions')
plt.title('Transactions per Christmas Period')

# Set x-ticks and labels
plt.xticks(x_values, christmas_transactions.index.astype(int))

# Remove grid lines
plt.grid(False)

plt.show()

### Hypothesis Testing

In [ ]:
# Finding the correlation between the sales and oil price
correlation = train['sales'].corr(train['oil_price'])

correlation


In [ ]:
# Plotting a Correlation Matrix
columns_to_analyze = ['sales', 'oil_price']
correlation_matrix = train[columns_to_analyze].corr()

# Plotting the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', square=True)
plt.title('Correlation Matrix')

plt.show()

There is a weak negative correlation between oil prices and sales which means oil prices do not have a significant effect on sales

#### This means we accept the Null Hypothesis that oil prices do not have an effect on sales

## Modeling 

### Feature Engineering 

#### Drop the unnecessary columns 
Drop id, locale, locale_name, description, transferred, state, store_type columns

In [ ]:
train = train.drop(columns = ['id', 'locale','locale_name','description','transferred','state','store_type','Year'], axis=1)

In [ ]:
train.columns

In [ ]:
time_data = train 

In [ ]:
time_data.head()

#### Standardization

In [ ]:
# create an instance of StandardScaler
scaler = StandardScaler()

# select numerical columns
num_cols = ['sales', 'transactions', 'oil_price']

# fit and transform the numerical columns
time_data[num_cols] = scaler.fit_transform(time_data[num_cols])


#### Feature Encoding

In [ ]:
time_data.info()

In [ ]:
# Select the categorical columns
categorical_columns = ["family", "city", "holiday_type"]
categorical_data = time_data[categorical_columns]

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the data to one hot encoding
one_hot_encoded_data = encoder.fit_transform(categorical_data)

# Get the categories for each column
categories = [encoder.categories_[i] for i in range(len(encoder.categories_))]

# Create the column names for the one hot encoded data
column_names = []
for i in range(len(categories)):
    for j in range(len(categories[i])):
        column_names.append(f'{categorical_columns[i]}_{categories[i][j]}')

# Convert the one hot encoding data to a DataFrame
one_hot_encoded_data = pd.DataFrame(one_hot_encoded_data.toarray(), columns=column_names)


# Reset the index of both dataframes
time_data = time_data.reset_index(drop=True)
one_hot_encoded_data = one_hot_encoded_data.reset_index(drop=True)

# Concatenate the original dataframe with the one hot encoded data
time_data_encoded = pd.concat([time_data, one_hot_encoded_data], axis=1)

# Drop the original categorical columns
time_data_encoded.drop(categorical_columns, axis=1, inplace=True)

In [ ]:
time_data_encoded.head()

In [ ]:
time_data_encoded = time_data_encoded.drop(columns = 'date', axis=1)

In [ ]:
time_data_encoded.head()

##### Split data

In [ ]:
# Create the feature dataframe using the selected columns
X = time_data_encoded.drop(["sales"], axis=1)

# Get the target variable
y = time_data_encoded.sales

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Decision Tree Regression Model

In [ ]:
# Decision Tree Regression Model
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

In [ ]:
# Make prediction on X_test
dt_predictions = dt.predict(X_test)
dt_predictions

In [ ]:
plt.scatter(y_test, dt_predictions)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("Decision Tree Regression")
plt.grid(False)
plt.show()

In [ ]:
# Evaluation Metrics for Decision Tree Regression
dt_mse = mean_squared_error(y_test, dt_predictions).round(2)
dt_rmse = np.sqrt(dt_mse).round(2)

In [ ]:
# Print the evaluation results for Decision Tree Regression model
print("\nEvaluation Results for Decision Tree Regression:")
print("MSE:", dt_mse)
print("RMSE:", dt_rmse)

#### Linear Regression

In [ ]:
# Linear Regression Model
lr = LinearRegression()
lr.fit(X_train, y_train)


In [ ]:
# Make prediction on X_test
lr_predictions = lr.predict(X_test)

In [ ]:
plt.scatter(y_test, lr_predictions)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("Linear Regression")
plt.grid(False)
plt.show()

In [ ]:
# Evaluation Metrics for Linear Regression
lr_mse = mean_squared_error(y_test, lr_predictions).round(2)
lr_rmse = np.sqrt(lr_mse).round(2)

In [ ]:
# Print the evaluation results for Linear Regression model
print("\nEvaluation Results for Linear Regression:")
print("MSE:", lr_mse)
print("RMSE:", lr_rmse)

#### Random Forest Regression Model 

In [ ]:
# Random Forest Regression Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
# Make prediction on X_test
rf_predictions = rf.predict(X_test)

In [ ]:
plt.scatter(y_test, rf_predictions)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("Random Forest Regression")
plt.grid(False)
plt.show()

In [ ]:
# Evaluation Metrics for Random Forest Regression
rf_mse = mean_squared_error(y_test, rf_predictions).round(2)
rf_rmse = np.sqrt(rf_mse).round(2)

In [ ]:
# Print the evaluation results for Random Forest Regrression model
print("\nEvaluation Results for Random Forest:")
print("MSE:", rf_mse)
print("RMSE:", rf_rmse)